Download VOC dataset

In [0]:
! wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012|/VOCtrainval_11-May-2012.tar

/bin/bash: /VOCtrainval_11-May-2012.tar: No such file or directory
--2020-03-28 05:13:44--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://host.robots.ox.ac.uk/pascal/VOC/voc2012/ [following]
--2020-03-28 05:13:45--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/
Reusing existing connection to host.robots.ox.ac.uk:80.
HTTP request sent, awaiting response... 200 OK
Length: 44877 (44K) [text/html]
Saving to: ‘voc2012’

voc2012             100%[===================>]  43.83K   275KB/s    in 0.2s    

2020-03-28 05:13:45 (275 KB/s) - ‘voc2012’ saved [44877/44877]



In [0]:
! tar -xvf VOCtrainval_11-May-2012.tar

tar: VOCtrainval_11-May-2012.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [0]:
! apt-get install caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libgflags2.2
  libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cuda
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-olefile python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pil
  python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyprocess python3-pygments python3-pyparsing python3-pywt
  python3-scipy python3-simplegeneric python3-six python3-skimage
  python3-skimage

In [0]:
! git clone https://github.com/jayantabh/adversarial-attacks.git

Cloning into 'adversarial-attacks'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 179 (delta 89), reused 97 (delta 51), pack-reused 40
Receiving objects: 100% (179/179), 4.49 MiB | 56.70 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [0]:
%cd adversarial-attacks/
%mkdir data/voc_val

/content/adversarial-attacks


In [0]:
%cp /content/VOCdevkit/VOC2012/JPEGImages/* /content/adversarial-attacks/data/voc_val/

cp: cannot stat '/content/VOCdevkit/VOC2012/JPEGImages/*': No such file or directory


In [0]:
%cp /content/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt /content/adversarial-attacks/data/voc_val.txt

cp: cannot stat '/content/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt': No such file or directory


In [0]:
%pwd

'/content/adversarial-attacks'

In [0]:
filenames = []
with open('/content/adversarial-attacks/data/voc_val.txt') as f:
    line = f.readline()
    filenames.append('data/voc_val/' + line[:-1] + '.jpg\n')
    cnt = 1

    while line:
        # print("Line {}: {}".format(cnt, line.strip()))
        line = f.readline()
        if len(line) > 0:
            filenames.append('data/voc_val/' + line[:-1] + '.jpg\n')
        cnt += 1

with open('/content/adversarial-attacks/data/voc_val_jpg.txt', 'w+') as w:
    w.writelines(filenames)

FileNotFoundError: ignored

In [0]:
! rm -rf /content/adversarial-attacks/output/voc_val/
# ! rm -rf /content/adversarial-attacks/experiments/

In [0]:
! bash /content/adversarial-attacks/experiments/experiment_base.sh

In [0]:
! bash examples/demo_cityscapes.sh

In [0]:
import numpy as np
import cv2
import os

In [0]:
%mkdir /content/adversarial-attacks/data/voc_val_target/
%cp /content/VOCdevkit/VOC2012/SegmentationClass/* /content/adversarial-attacks/data/voc_val_target/

In [0]:
def iou_ratio(filenames='data/voc_val_jpg.txt', op_dir='/content/adversarial-attacks/output/voc_val/', target_dir='data/voc_val_target', eps=8, target_idx=0, model_name='fcn8'):
    lines = []
    
    with open(filenames) as f:
        line = f.readline()
        lines.append(line)

        while line:
            line = f.readline()

            if len(line) > 0:
                lines.append(line)

    # target_dir = op_dir + '_target'
    iou_sum = 0
    num_sam = 0

    for line in lines:
        temp = line.split('/')
        fname = temp[-1].split('.')[0]

        adv_pred = fname + '_adversarial_pred_' + model_name + '_eps=' + str(eps) + '_target_idx=' + str(target_idx) + '.png'
        orig_pred = fname + '_original_pred_' + model_name + '.png'
        target = fname + '.png'

        adv_path = os.path.join(op_dir, adv_pred)
        orig_path = os.path.join(op_dir, orig_pred)
        tar_path = os.path.join(target_dir, target)
        
        if not os.path.exists(adv_path):
            print(adv_path)
        if not os.path.exists(orig_path):
            print(orig_path)
        if not os.path.exists(tar_path):
            print(tar_path)

        im_adv = cv2.imread(adv_path)
        im_orig = cv2.imread(orig_path)
        im_tar = cv2.imread(tar_path)

        # print(im_adv)
        # print(im_orig)
        # print(im_tar)

        adv_tar_int = np.logical_and(np.logical_and(im_tar == im_adv, im_tar != 0), im_adv != 0).astype(np.int).sum() + 1e-8
        adv_tar_un = np.logical_or(im_tar != 0, im_adv != 0).astype(np.int).sum() + 1e-8
        orig_tar_int = np.logical_and(np.logical_and(im_tar == im_orig, im_tar != 0), im_orig != 0).astype(np.int).sum() + 1e-8
        orig_tar_un = np.logical_or(im_tar != 0, im_orig != 0).astype(np.int).sum() + 1e-8

        if adv_tar_int/adv_tar_un < orig_tar_int/orig_tar_un:
            iou_sum += ((adv_tar_int/adv_tar_un)/(orig_tar_int/orig_tar_un))    
            num_sam += 1

    return iou_sum / num_sam


In [0]:
import csv

def iou_ratio(filenames='data/voc_val_jpg.txt', 
              op_dir='/content/adversarial-attacks/output/voc_val/', 
              target_dir='data/voc_val_target', 
              iou_file='data/voc_val_iou.csv', 
              eps=8, target_idx=0, model_name='fcn8'):
    lines = []
    
    with open(filenames) as f:
        line = f.readline()
        lines.append(line)

        while line:
            line = f.readline()

            if len(line) > 0:
                lines.append(line)

    # target_dir = op_dir + '_target'
    iou_sum = 0
    num_sam = 0

    with open(iou_file, 'w+') as ifile:
        csv_writer = csv.writer(ifile)
        for line in lines:
            temp = line.split('/')
            fname = temp[-1].split('.')[0]

            adv_pred = fname + '_adversarial_pred_' + model_name + '_eps=' + str(eps) + '_target_idx=' + str(target_idx) + '.png'
            orig_pred = fname + '_original_pred_' + model_name + '.png'
            target = fname + '.png'

            adv_path = os.path.join(op_dir, adv_pred)
            orig_path = os.path.join(op_dir, orig_pred)
            tar_path = os.path.join(target_dir, target)
            
            if not os.path.exists(adv_path):
                print(adv_path)
            if not os.path.exists(orig_path):
                print(orig_path)
            if not os.path.exists(tar_path):
                print(tar_path)

            im_adv = cv2.imread(adv_path)
            im_orig = cv2.imread(orig_path)
            im_tar = cv2.imread(tar_path)

            # print(im_adv)
            # print(im_orig)
            # print(im_tar)

            adv_tar_int = np.logical_and(np.logical_and(im_tar == im_adv, im_tar != 0), im_adv != 0).astype(np.int).sum() + 1e-8
            adv_tar_un = np.logical_or(im_tar != 0, im_adv != 0).astype(np.int).sum() + 1e-8
            orig_tar_int = np.logical_and(np.logical_and(im_tar == im_orig, im_tar != 0), im_orig != 0).astype(np.int).sum() + 1e-8
            orig_tar_un = np.logical_or(im_tar != 0, im_orig != 0).astype(np.int).sum() + 1e-8

            iou = ((adv_tar_int/adv_tar_un)/(orig_tar_int/orig_tar_un))

            print(iou)

            if adv_tar_int/adv_tar_un < orig_tar_int/orig_tar_un:
                iou_row = [fname, model_name, str(eps), iou]
                csv_writer.writerow(iou_row)
                iou_sum += iou    
                num_sam += 1

    return iou_sum / num_sam

In [0]:
iou_ratio(eps=2.0, model_name='dilated_context') #2007_000033_adversarial_pred_fcn8_eps=8.0_target_idx=0.png 2007_000033_adversarial_pred_fcn8_eps=0.25_target_idx=0.png

In [0]:
iou_ratio(eps=32.0)

In [0]:
! wget -N http://liangchiehchen.com/projects/released/deeplab_aspp_vgg16/prototxt_and_model.zip 

In [0]:
%cp prototxt_and_model.zip models/

In [0]:
! find / -name "*layers*"

In [0]:
! mv /content/interp_layer.cpp /usr/local/lib/python3.6/dist-packages/caffe2/python/layers/

In [0]:
! ls -la /usr/local/lib/python3.6/dist-packages/caffe2/python/layers/ | grep interp

In [0]:
! cp /content/interp_layer.cpp /usr/local/lib/python2.7/dist-packages/caffe2/python/layers

In [0]:
! zip -r output.zip /content/adversarial-attacks/output

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%ls -la output/voc_val/

In [0]:
%cp output.zip /content/drive/My\ Drive/output_1.zip